In [1]:
import json
import pandas as pd
import numpy as np
import random

### Load Spotify Data

In [2]:
# Reading a JSON file
with open('spotify_data.json', 'r') as f:
    data = json.load(f)

### Data of Interest

This is the data of interest. For our labels, we need the genre of each track, for out Input arrays we use pitch and timbre values of each segment.

In [3]:
# Genre tag
data[0]['Genre']

'Rock'

In [4]:
# Access to pitch vector of a segment of a track
data[0]['segments'][0]['pitches']

[1.0,
 0.908,
 0.039,
 0.032,
 0.028,
 0.025,
 0.021,
 0.022,
 0.023,
 0.014,
 0.008,
 0.07]

In [5]:
# Access to timbre vector of a segment of a track
data[0]['segments'][0]['timbre']

[1.328,
 152.697,
 16.436,
 -61.842,
 48.839,
 -61.504,
 28.814,
 -28.79,
 -30.275,
 -2.638,
 11.721,
 -9.813]

### Sample balancing

In [6]:
genres = []
for i in data:
    genre = i['Genre']
    genres.append(genre)

y = np.array(genres)

In [7]:
# Count unique occurrences of each genre in response vector
y_df = pd.DataFrame(y)
y_df[0].value_counts()

Jazz         1414
Metal        1404
Country      1363
Classical    1259
Rock         1213
Hip-Hop      1208
Pop          1153
Blues        1101
Name: 0, dtype: int64

Sample is unbalanced. Balance by dropping out some observations (downsampling by resampling without replacement)

In [8]:
# Balancing
datarock = []
datapop = []
datacountry = []
dataclassic = []
datametal = []
datahiphop = []
datablues = []
datajazz = []


for item in data:
    if item['Genre'] == 'Rock':
        datarock.append(item)
    elif item['Genre'] == 'Pop':
        datapop.append(item)
    elif item['Genre'] == 'Country':
        datacountry.append(item)
    elif item['Genre'] == 'Classical':
        dataclassic.append(item)
    elif item['Genre'] == 'Metal':
        datametal.append(item)
    elif item['Genre'] == 'Hip-Hop':
        datahiphop.append(item)
    elif item['Genre'] == 'Blues':
        datablues.append(item)
    elif item['Genre'] == 'Jazz':
        datajazz.append(item)

random.shuffle(datarock)
random.shuffle(datapop)
random.shuffle(datacountry)
random.shuffle(dataclassic)
random.shuffle(datahiphop)
random.shuffle(datametal)
random.shuffle(datajazz)
random.shuffle(datablues)

datatrainrock = datarock[:880]
datatrainpop = datapop[:880]
datatraincountry = datacountry[:880]
datatrainclassic = dataclassic[:880]
datatrainmetal = datametal[:880]
datatrainhiphop = datahiphop[:880]
datatrainblues = datablues[:880]
datatrainjazz = datajazz[:880]

datavalrock = datarock[880:990]
datavalpop = datapop[880:990]
datavalcountry = datacountry[880:990]
datavalclassic = dataclassic[880:990]
datavalmetal = datametal[880:990]
datavalhiphop = datahiphop[880:990]
datavalblues = datablues[880:990]
datavaljazz = datajazz[880:990]

datatestrock = datarock[990:1100]
datatestpop = datapop[990:1100]
datatestcountry = datacountry[990:1100]
datatestclassic = dataclassic[990:1100]
datatestmetal = datametal[990:1100]
datatesthiphop = datahiphop[990:1100]
datatestblues = datablues[990:1100]
datatestjazz = datajazz[990:1100]

datatrain = datatrainrock + datatrainpop + datatrainhiphop + datatrainclassic + datatraincountry + datatrainmetal + datatrainjazz + datatrainblues
dataval = datavalrock + datavalpop + datavalhiphop + datavalclassic + datavalcountry + datavalmetal + datavaljazz + datavalblues
datatest = datatestrock + datatestpop + datatesthiphop + datatestclassic + datatestcountry + datatestmetal + datatestjazz + datatestblues

In [37]:
# Shuffle
random.shuffle(datatrain)
random.shuffle(dataval)
random.shuffle(datatest)

In [9]:
# Combine
data_b = datatrain + dataval + datatest

### Create Input arrays

In [67]:
# No. of segments to include for each track
segs = 512

#### Train 

In [68]:
# Create Input Arrays
N = len(datatrain)

# Create empty array with appropriate dimensions
X_Train = np.empty((N, segs, 12, 2))

# Input array with timbre + pitch
# Combine pitches and timbre vectors of each track, each segment into numpy array
for i in range(len(datatrain)):
    for j in range(len(datatrain[i]['segments'][:segs])):
        timbre = datatrain[i]['segments'][j]['timbre']
        pitch = datatrain[i]['segments'][j]['pitches']
        mat = np.append([timbre], [pitch], axis=0)
        X_Train[i][j] = mat.T

# Input array timbre only
Xt_Train = np.empty((N, segs, 12))
for i in range(len(datatrain)):
    for j in range(len(datatrain[i]['segments'][:segs])):
        timbre = np.array(datatrain[i]['segments'][j]['timbre'])
        Xt_Train[i][j] = timbre

# Input array pitch only
Xp_Train = np.empty((N, segs, 12))
for i in range(len(datatrain)):
    for j in range(len(datatrain[i]['segments'][:segs])):
        pitch = np.array(datatrain[i]['segments'][j]['pitches'])
        Xp_Train[i][j] = pitch

#### Validation

In [69]:
# Create Input Arrays
N = len(dataval)

# Create empty array with appropriate dimensions
X_Val = np.empty((N, segs, 12, 2))

# Input array with timbre + pitch
# Combine pitches and timbre vectors of each track, each segment into numpy array
for i in range(len(dataval)):
    for j in range(len(dataval[i]['segments'][:segs])):
        timbre = dataval[i]['segments'][j]['timbre']
        pitch = dataval[i]['segments'][j]['pitches']
        mat = np.append([timbre], [pitch], axis=0)
        X_Val[i][j] = mat.T

# Input array timbre only
Xt_Val = np.empty((N, segs, 12))
for i in range(len(dataval)):
    for j in range(len(dataval[i]['segments'][:segs])):
        timbre = np.array(dataval[i]['segments'][j]['timbre'])
        Xt_Val[i][j] = timbre

# Input array pitch only
Xp_Val = np.empty((N, segs, 12))
for i in range(len(dataval)):
    for j in range(len(dataval[i]['segments'][:segs])):
        pitch = np.array(dataval[i]['segments'][j]['pitches'])
        Xp_Val[i][j] = pitch

#### Test

In [70]:
# Create Input Arrays
N = len(datatest)

# Create empty array with appropriate dimensions
X_Test = np.empty((N, segs, 12, 2))

# Input array with timbre + pitch
# Combine pitches and timbre vectors of each track, each segment into numpy array
for i in range(len(datatest)):
    for j in range(len(datatest[i]['segments'][:segs])):
        timbre = datatest[i]['segments'][j]['timbre']
        pitch = datatest[i]['segments'][j]['pitches']
        mat = np.append([timbre], [pitch], axis=0)
        X_Test[i][j] = mat.T

# Input array timbre only
Xt_Test = np.empty((N, segs, 12))
for i in range(len(datatest)):
    for j in range(len(datatest[i]['segments'][:segs])):
        timbre = np.array(datatest[i]['segments'][j]['timbre'])
        Xt_Test[i][j] = timbre

# Input array pitch only
Xp_Test = np.empty((N, segs, 12))
for i in range(len(datatest)):
    for j in range(len(datatest[i]['segments'][:segs])):
        pitch = np.array(datatest[i]['segments'][j]['pitches'])
        Xp_Test[i][j] = pitch

### Labels

In [71]:
genre = []
for i in datatrain:
    genre = i['Genre']
    genre.append(genre)

Y_Train = np.array(genre)

In [72]:
genre = []
for i in dataval:
    genre = i['Genre']
    genre.append(genre)

Y_Val = np.array(genres)

In [73]:
genre = []
for i in datatest:
    genre = i['Genre']
    genre.append(genre)

Y_Test = np.array(genre)

In [74]:
# Verify our sample is balanced
pd.DataFrame(Y_Train)[0].value_counts(),pd.DataFrame(Y_Val)[0].value_counts(),pd.DataFrame(Y_Test)[0].value_counts()

(Jazz         880
 Blues        880
 Classical    880
 Hip-Hop      880
 Pop          880
 Country      880
 Rock         880
 Metal        880
 Name: 0, dtype: int64,
 Jazz         110
 Hip-Hop      110
 Country      110
 Rock         110
 Classical    110
 Metal        110
 Pop          110
 Blues        110
 Name: 0, dtype: int64,
 Jazz         110
 Hip-Hop      110
 Country      110
 Rock         110
 Classical    110
 Metal        110
 Pop          110
 Blues        110
 Name: 0, dtype: int64)

==> Sample is balanced

### Save compressed NumPy Arrays

In [75]:
# Save compressed NumPy Arrays
np.savez_compressed('orpheusC' + str(segs) + '.npz', 
                    X_Train=X_Train, Xt_Train=Xt_Train, Xp_Train=Xp_Train, 
                    X_Val=X_Val, Xt_Val=Xt_Val, Xp_Val=Xp_Val, 
                    X_Test=X_Test, Xt_Test=Xt_Test, Xp_Test=Xp_Test, 
                    Y_Train=Y_Train, Y_Val=Y_Val, Y_Test=Y_Test)